In [1]:
import pyvista as pv
import numpy as np
from pathlib import Path

In [2]:
file = Path('blood/centerlines.vtp')
cl = pv.read(file)

In [3]:
cl.n_cells, cl.n_lines, cl.n_arrays, cl.n_points

(6, 6, 1, 701)

In [4]:
cl2 = cl.compute_arc_length()
cl2['arc_length'].shape

(701,)

In [5]:
cl.head()

'PolyData (0x11b1908e0)\n  N Cells:\t6\n  N Points:\t701\n  X Bounds:\t-4.257e+01, 2.567e+01\n  Y Bounds:\t-1.400e+00, 4.615e+01\n  Z Bounds:\t-1.812e+01, 9.256e+01\n  N Arrays:\t1\n'

In [6]:
npoints = [cl.cell_n_points(i) for i in range(cl.n_cells)]
start = np.cumsum([0]+npoints)
lines = [np.arange(start[i],start[i+1]) for i in range(len(start)-1)]
cl.compute_arc_length()
cl.cell_data

pyvista DataSetAttributes
Association     : CELL
Active Scalars  : None
Active Vectors  : None
Active Texture  : None
Active Normals  : None
Contains arrays : None

In [7]:
p = []
[p.extend(cl.cell_points(i)) for i in range(cl.n_cells)]
pts = np.array(p)

In [8]:
from scipy.spatial import KDTree

In [9]:
def remove_close_points(pts, r=0.001):
    pairs = KDTree(pts).query_pairs(r=r)
    remove = list(set([b for a,b in pairs]))
    keep = np.ones(len(pts))
    keep[remove] = 0
    return pts[keep.astype(bool)]

In [10]:
def find_closest_pair(pts):
    dist_idx = KDTree(pts).query(pts)
    

In [11]:
pts2 = remove_close_points(pts)
pts3 = remove_close_points(pts2, r=0.1)
for i in range(2, 10):
    print(i*0.1)


0.2
0.30000000000000004
0.4
0.5
0.6000000000000001
0.7000000000000001
0.8
0.9


In [12]:
lines = [pv.PolyData(cl.cell_points(i))  for i in range(cl.n_cells)]
#length = [l['arc_length'] for l in lines]

In [13]:
lines[0]

PolyData,Information
N Cells,89
N Points,89
X Bounds,"-2.838e+01, -1.971e+01"
Y Bounds,"2.862e+01, 3.915e+01"
Z Bounds,"-1.812e+01, 5.463e+01"
N Arrays,0


In [14]:
A = np.arange(12).reshape(2,6)

In [15]:
A[0,:]

array([0, 1, 2, 3, 4, 5])

In [20]:
%run PythonScripts/distance_map_v2.py blood/mesh/fontan_stor_v5_L1_bl050_sl3.vtu blood/centerlines.vtp 0.5 2

  Creating uniform grid of dimension [155 118 240]
  Calculating distance from grid nodes to the surface ...
  Creating unstructured grid with outer wall of 2 voxels ...
  Creating tubes ...
  Check for unconnected tubes ...
  Unconnected tube 5 is merged into solid tube 1 ...
  Calculate slice indices ...
min, max, c : 0, 136, 137
min, max, c : 122, 163, 179
min, max, c : 70, 88, 198
min, max, c : 83, 125, 241
ugrid.n_cells: 397513, ugrid2.n_cells: 397513
  Saved blood/mesh/distance_map.vtk


In [17]:
nproc = 10
proc_size = np.ceil(tube.size/nproc)

AttributeError: 'list' object has no attribute 'size'

In [ ]:
mask = slice<10
print(np.sum(slice<10))
print(tube[mask].min(), tube[mask].max())

16830
2 2


In [ ]:
mask = slice<1
tube

array([2, 2, 2, ..., 3, 3, 3])

In [ ]:
slice

array([  0,   0,   0, ..., 178, 178, 178])

In [ ]:
proc = -1*ones(ugrid.n_cells, dtype=int)
stop = npmax(slice)
a = b = 0
p = 0
while b < stop:
    mask = (slice>=a)*(slice<=b)
    if np.sum(mask) >= proc_size:
        print(f'p: {p}, a,b: {a},{b}')
        a = b
        proc[mask] = p
        p += 1
    b += 1
ugrid['proc'] = proc
ugrid.save('processes.vtk')        

p: 0, a,b: 0,23
p: 1, a,b: 23,45
p: 2, a,b: 45,61
p: 3, a,b: 61,73
p: 4, a,b: 73,86
p: 5, a,b: 86,110
p: 6, a,b: 110,149
p: 7, a,b: 149,223
p: 8, a,b: 223,242
p: 9, a,b: 242,262


In [ ]:
tgrid5 = ugrid.threshold(value=5, scalars='tube')

In [ ]:
conn = tgrid5.connectivity()

In [ ]:
bodies = conn.split_bodies()

In [ ]:
vol = [bodies[k].volume for k in bodies.keys()]

In [ ]:
body_list = [bodies[bodies.keys()[i]] for i in np.argsort(vol)]

In [ ]:
tgrid1 = ugrid.threshold(value=(1,1), scalars='tube')

In [ ]:
t51 = tgrid5 + tgrid1

In [ ]:
to_move = pv.UnstructuredGrid().merge(body_list[:-1])

In [ ]:
sel = ugrid.select_enclosed_points(to_move.extract_surface(), tolerance=0.0, check_surface=False)
mask = sel.point_data['SelectedPoints'].view(bool)

In [ ]:
sel.plot(scalars='tube')

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [ ]:
ugrid

UnstructuredGrid (0x1290b0220)
  N Cells:	397513
  N Points:	434719
  X Bounds:	-4.602e+01, 3.148e+01
  Y Bounds:	-6.775e+00, 5.173e+01
  Z Bounds:	-2.208e+01, 9.742e+01
  N Arrays:	11

In [ ]:
sel

UnstructuredGrid (0x12912f4c0)
  N Cells:	397513
  N Points:	434719
  X Bounds:	-4.602e+01, 3.148e+01
  Y Bounds:	-6.775e+00, 5.173e+01
  Z Bounds:	-2.208e+01, 9.742e+01
  N Arrays:	12

In [ ]:
ugrid2 = sel.point_data_to_cell_data('SelectedPoints')

In [ ]:
ugrid2['tube'][ugrid2['SelectedPoints']>0] = -1

In [ ]:
ugrid2.plot(scalars='tube')

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [ ]:
to_move['tube'] = -1*np.ones(to_move.n_cells, dtype=int)

In [ ]:
grid3 = ugrid + to_move

In [ ]:
to_move.plot(scalars='tube')

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [ ]:
sum([b.n_cells for b in body_list][:-1])

1063

In [ ]:
397513-396450

1063